In [67]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
from datetime import date, datetime,timedelta
import openpyxl
from openpyxl import Workbook
import os
import openpyxl.workbook
import pandas as pd

reservas = []

def fazer_reserva(data):
    fonte_titulo = ("Arial",24,"bold")
    fonte_botoes = ("Arial",20)

    janela_reserva = tk.Toplevel()
    janela_reserva.title("Fazer Reserva")
    janela_reserva.configure(bg="#ffffff")
    
    frame_title = tk.Frame(janela_reserva,bg="#ffffff")
    frame_title.pack(pady=10)
    tk.Label(frame_title,
            text="Fazer Reserva",
            font=fonte_titulo,bg="#ffffff").pack()
    
    frame_formulario = tk.Frame(janela_reserva,
    bg="#ffffff")
    frame_formulario.pack(pady=10)
    
    data_label = tk.Label(frame_formulario,
                        text="Digite a data da reserva (dd/mm/aaaa): ",
                        font=fonte_titulo,bg="#ffffff")
    data_label.grid(row=0,column=0,pady=10,sticky='e')
    data_entry = tk.Entry(frame_formulario,font=("Arial",18))
    data_entry.grid(row=0,column=1,pady=10,padx=10)
    data_entry.insert(0,data)

    hora_inicio_label = tk.Label(frame_formulario,
                        text="Digite a hora inicial da reserva (Entre 09:00 e 23:00): ",
                        font=fonte_titulo,bg="#ffffff")
    hora_inicio_label.grid(row=1,column=0,pady=10,sticky='e')
    hora_inicio_entry = tk.Entry(frame_formulario,font=("Arial",18))
    hora_inicio_entry.grid(row=1,column=1,padx=10,pady=10)

    hora_fim_label = tk.Label(frame_formulario,
                        text="Digite a hora final da reserva (Entre 09:00 e 23:00): ",
                        font=fonte_titulo,bg="#ffffff")
    hora_fim_label.grid(row=2,column=0,pady=10,sticky='e')
    hora_fim_entry = tk.Entry(frame_formulario,font=("Arial",18))
    hora_fim_entry.grid(row=2,column=1,padx=10,pady=10)

    cpf_label = tk.Label(frame_formulario,
                        text="Digite o CPF: ",
                        font=fonte_titulo,bg="#ffffff")
    cpf_label.grid(row=3,column=0,pady=10,sticky='e')
    cpf_entry = tk.Entry(frame_formulario,font=("Arial",18))
    cpf_entry.grid(row=3,column=1,padx=10,pady=10)

    nome_label = tk.Label(frame_formulario,
                        text="Digite o Nome: ",
                        font=fonte_titulo,bg="#ffffff")
    nome_label.grid(row=4,column=0,pady=10,sticky='e')
    nome_entry = tk.Entry(frame_formulario,font=("Arial",18))
    nome_entry.grid(row=4,column=1,padx=10,pady=10)

    def preecher_valor():
        hora_inicio = hora_inicio_entry.get()
        hora_fim = hora_fim_entry.get()

        valor_total = calcular_valor(hora_inicio,hora_fim)
        valor_entry.delete(0,tk.END)
        valor_entry.insert(0,valor_total)

    botao_calcular_valor = tk.Button(frame_formulario,
                                    text="Calcular Valor",
                                    font=fonte_botoes,
                                    command=preecher_valor)
    botao_calcular_valor.grid(row=5,column=0,pady=10,sticky='e')
    valor_entry = tk.Entry(frame_formulario,font=("Arial",18))
    valor_entry.grid(row=5,column=1,padx=10,pady=10)

    def calcular_valor(hora_inicio,hora_fim):
        hora_inicio_str = hora_inicio[:-3]
        minuto_inicio_str = hora_inicio[-2:]    
        hora_final_str = hora_fim[:-3]
        minuto_final_str = hora_fim[-2:]

        duracao_minutos = (int(hora_final_str) * 60 + int(minuto_final_str)) - (int(hora_inicio_str) *60 + int(minuto_inicio_str))
        valor_total = duracao_minutos * 8 /60  

        return valor_total

    def confirmar_reserva():
        data = data_entry.get()
        hora_inicio = hora_inicio_entry.get()
        hora_final = hora_fim_entry.get()
        cpf = cpf_entry.get()
        nome = nome_entry.get()
        valor = float(valor_entry.get())

        if hora_inicio < "09:00" or hora_final > "23:00" or hora_inicio >= hora_final:
            messagebox.showerror("Error","Horário invalido. Tente novamente")
            return
        if verificar_conflito_reserva(hora_inicio,hora_final):
            messagebox.showerror("Error","Já existe uma reserva neste horário.Escolha outro horário")
            return

        caminho = "Dados.xlsx"
        planilha = "Reservas"

        try:
            workbook = openpyxl.load_workbook(caminho)
            sheet = workbook[planilha]
        except FileExistsError:
            print("Arquivo não encontrado!")
        
        hora_inicio_str = hora_inicio[:-3]
        minuto_inicio_str = hora_inicio[-2:]    
        hora_final_str = hora_final[:-3]
        minuto_final_str = hora_final[-2:]

        duracao_minutos = (int(hora_final_str) * 60 + int(minuto_final_str)) - (int(hora_inicio_str) *60 + int(minuto_inicio_str))
        valor_total = duracao_minutos * 8 /60
        sheet.append((data,hora_inicio,hora_final,cpf,nome,valor))
        workbook.save(caminho)

        messagebox.showinfo("Sucesso",f"A reserva das {hora_inicio_str}:{minuto_inicio_str}"+
        f" ás {hora_final_str}:{minuto_final_str}, relizada com sucesso! \n O valor total da reserva é de R${valor_total:.2f}")
        janela_reserva.destroy()

    botao_confirmar = tk.Button(janela_reserva,
    text="Confirmar",font=fonte_botoes,command=confirmar_reserva)
    botao_confirmar.pack(pady=10)

    janela_reserva.update_idletasks()

    largura = janela_reserva.winfo_width()
    altura = janela_reserva.winfo_height()

    pos_x = (janela_reserva.winfo_screenwidth() // 2) - (largura // 2)
    pos_y = (janela_reserva.winfo_screenheight() // 2) - (altura // 2)

    janela_reserva.geometry(f"{largura}x{altura}+{pos_x}+{pos_y}")   

    def verificar_conflito_reserva(hora_inicio,hora_final):
        caminho = "Dados.xlsx"
        planilha = "Reservas"

        try:
            workbook = openpyxl.load_workbook(caminho)
            sheet = workbook[planilha]
        except FileExistsError:
            return False
                
        for row in sheet.iter_rows(min_row=2,values_only=True):
            reserva_data = row[0]
            reserva_hora_inicio = row[1]
            reserva_hora_fim = row[2]
            if data == reserva_data and hora_inicio < reserva_hora_fim and hora_final > reserva_hora_inicio:
                return True

        return False

def ver_reserva():
    caminho = "Dados.xlsx"
    planilha = "Reservas"

    try:
        workbook = openpyxl.load_workbook(caminho)
        sheet = workbook[planilha]
    except FileExistsError:
        print("Arquvio não encontrado!")
        return
                
    fonte_titulo = ("Arial",24,"bold")
    fonte_botoes = ("Arial",20)
    janela_reservas = tk.Toplevel()
    janela_reservas.title("Ver Reservas")
    janela_reservas.configure(bg="#ffffff")
    
    tk.Label(janela_reservas,
            text="Visualização de Reservas",
            font=fonte_titulo,bg="#ffffff").pack(pady=10)

    frame_painel = tk.Frame(janela_reservas,bg="#ffffff")
    frame_painel.pack(pady=10)

    largura_canvas = 950
    canvas = tk.Canvas(frame_painel,bg="#ffffff",width=largura_canvas)
    canvas.pack(sid="left",fill="both",expand=True)

    scroll_bar = tk.Scrollbar(frame_painel,orient="vertical",command=canvas.yview)
    scroll_bar.pack(side="right",fill="y")

    canvas.configure(yscrollcommand=scroll_bar.set)
    
    canvas.bind("<Configure>",lambda e: canvas.configure(yscrollregion=canvas.bbox("all")))

    frame_horarios = tk.Frame(canvas,bg="#ffffff")
    canvas.create_window((0,0),window=frame_horarios,anchor="nw")

    frame_filtrar = tk.Frame(janela_reservas,bg="#ffffff")
    frame_filtrar.pack(pady=10)

    label_data_filtrar = tk.Label(frame_filtrar,text="Data: ",font=("Arial",16),bg="#ffffff")
    label_data_filtrar.grid(row=0,column=0,padx=10,sticky="e")
    entry_data_filtrar = tk.Entry(frame_filtrar,font=("Arial",16),width=20)
    entry_data_filtrar.grid(row=0,column=1,pady=10) 
    data_atual = date.today().strftime("%d/%m/%Y")
    entry_data_filtrar.insert(0,data_atual) 
    
    def filtrar_data():
        data_filtrada = entry_data_filtrar.get()

        df = pd.read_excel("Dados.xlsx")
        reservas_filtradas = df[df['Data'] == data_filtrada]
        reservas = []
        
        for _,row in reservas_filtradas.iterrows():
            hora_inicio = row["Hora Inicio"]
            hora_fim = row["Hora Fim"]
            reservas.append({"Hora Inicio":hora_inicio,"Hora Fim":hora_fim})
        
        preencher_horarios(reservas)

    botao_data_filtrar = tk.Button(frame_filtrar,text="Filtrar",
    font=("Arial",16),command=filtrar_data)
    botao_data_filtrar.grid(row=0,column=2,pady=10,padx=10)
    botao_reserva_filtrar = tk.Button(frame_filtrar,text="Reservas",
    font=("Arial",16),command=lambda: fazer_reserva(entry_data_filtrar.get()))
    botao_reserva_filtrar.grid(row=0,column=3,pady=10)

    def incrementar_horario(horario):
        dt = datetime.strptime(horario,"%H:%M")
        dt += timedelta(minutes=30)
        return dt.strftime("%H:%M")

    def preencher_horarios(reservas=None):
        for child in frame_horarios.winfo_children():
            child.destroy()
        
        horario = "09:00"

        row = 0
        row_2 = 0

        while horario <= "23:00":
            reservado = False

            if reservas is not None:
                for reserva in reservas:
                    horario_inicio = str(reserva["Hora Inicio"])
                    horario_fim = str(reserva["Hora Fim"])
                    if horario_inicio <= horario < horario_fim:
                        reservado = True
                        break

            if "16:30" >= horario >="09:00":
                if reservado:
                    label_status = tk.Label(frame_horarios,
                    text=f"das {horario} às {incrementar_horario(horario)}",
                    font=("Arial",16),bg="#ffff00",width=20,padx=50)
                else:
                    label_status = tk.Label(frame_horarios,
                    text=f"das {horario} às {incrementar_horario(horario)}",
                    font=("Arial",16),bg="#00ff00",width=20,padx=50)

                if horario <= "16:30":
                    label_status.grid(row=row,column=0,pady=5,padx=50,sticky="w")
                else:
                    label_status.grid(row=row,column=1,pady=5,padx=50,sticky="w")

                row += 1

            elif "23:00" >= horario >="16:30":
                if reservado:
                    label_status = tk.Label(frame_horarios,
                    text=f"das {horario} às {incrementar_horario(horario)}",
                    font=("Arial",16),bg="#00ff00",width=20,padx=50)
                else:
                    label_status = tk.Label(frame_horarios,
                    text=f"das {horario} às {incrementar_horario(horario)}",
                    font=("Arial",16),bg="#00ff00",width=20,padx=50)

                if horario <= "23:00":
                    label_status.grid(row=row_2,column=2,pady=5,padx=50,sticky="w")
                else:
                    label_status.grid(row=row_2,column=3,pady=5,padx=50,sticky="w")

                row_2 += 1

            horario = incrementar_horario(horario)

        canvas.update_idletasks()
        canvas.configure(scrollregion=canvas.bbox("all"))

    filtrar_data()

    janela_reservas.update_idletasks()

    largura = janela_reservas.winfo_width()
    altura = janela_reservas.winfo_height()

    pos_x = (janela_reservas.winfo_screenwidth() // 2) - (largura // 2)
    pos_y = (janela_reservas.winfo_screenheight() // 2) - (altura // 2)

    janela_reservas.geometry(f"{largura}x{altura}+{pos_x}+{pos_y}")

def detalhe_reserva():
    fonte_titulo = ("Arial",24,"bold")
    fonte_botoes = ("Arial",20)
    janela_detalhe = tk.Toplevel()
    janela_detalhe.title("Detalhe de Reservas")
    janela_detalhe.configure(bg="#ffffff")

    frame_entries = tk.Frame(janela_detalhe,bg="#ffffff")
    frame_entries.pack(pady=10)

    campos = ["Data","Hora Inicio","Hora Fim","CPF","Nome","Preço"]
    entries = {}

    for i,campo in enumerate(campos):
        frame = tk.Frame(frame_entries,bg="#ffffff")
        frame.grid(row=i // 3, column=i % 3,padx=10,pady=5)

        label = tk.Label(frame,text=campo + ":",font=("Arial",14),bg="#ffffff")
        label.pack(side="left")
        entry = tk.Entry(frame,font=("Arial",14))
        entry.pack(side="left")
        entries[campo] = entry

    frame_soma = tk.Frame(janela_detalhe,bg="#ffffff")
    frame_soma.pack(pady=10)

    label_soma = tk.Label(frame_soma,text="Soma da coluna Preço: ",font=("Arial",14),bg="#ffffff")
    label_soma.pack(side="left",padx=10)
    
    label_valor_soma = tk.Label(frame_soma,text="0.00",font=("Arial",14),bg="#ffffff")
    label_valor_soma.pack(side="left",padx=10)
    
    label_status = tk.Label(frame_soma,text="",font=("Arial",14),bg="#ffffff")
    label_status.pack()

    frame_opcoes = tk.Frame(janela_detalhe,bg="#ffffff")
    frame_opcoes.pack(pady=10)

    frame_datas = tk.Frame(janela_detalhe,bg="#ffffff")
    frame_datas.pack(pady=10)

    label_data_inicio = tk.Label(frame_datas,text="Data Inicio: ",font=("Arial",14),bg="#ffffff")
    label_data_inicio.pack(side="left",padx=10)
    entry_data_inicio = tk.Entry(frame_datas,font=("Arial",14),bg="#ffffff")
    entry_data_inicio.pack(side="left",padx=10)

    label_data_fim = tk.Label(frame_datas,text="Data Fim: ",font=("Arial",14),bg="#ffffff")
    label_data_fim.pack(side="left",padx=10)
    entry_data_fim = tk.Entry(frame_datas,font=("Arial",14),bg="#ffffff")
    entry_data_fim.pack(side="left",padx=10)

    def filtrar():
        for linha in tree.get_children():
            tree.delete(linha)
        
        filtro = "Entre Datas"
        data_inicio = entry_data_inicio.get()
        data_fim =  entry_data_fim.get()

        soma_preco = 0.0
        for row in dados[1:]:
            if(not data_inicio or not data_fim) or (data_inicio <= row[0] <= data_fim):
                if all(entry.get().lower() in str(value).lower() for entry,value in zip(entries.values(),row)):
                    tree.insert("",tk.END,values=row)
                    soma_preco += row[-1]
        
        label_valor_soma.config(text=f"{soma_preco:.2f}")

    botao_filtrar = tk.Button(janela_detalhe,
    font=("Arial",14),text="Filtrar",command=filtrar,width=90)
    botao_filtrar.pack(padx=10,pady=10)

    tree = ttk.Treeview(janela_detalhe,show='headings')
    tree["columns"] = tuple(campos)

    for campo in campos:
        tree.heading(campo,text=campo,anchor=tk.CENTER)
        tree.column(campo, anchor=tk.CENTER,width=100)

    caminho = "Dados.xlsx"
    panilha = "Reservas"
    try:
        workbook = openpyxl.load_workbook(caminho)
        sheet = workbook[panilha]

    except FileExistsError:
        messagebox.showerror("Erro","Arquivo dados não foi encontrado!")
        janela_detalhe.destroy()
        return

    dados = []
    for row in sheet.iter_rows(values_only=True):
        dados.append(row)

    scrollbar_y = ttk.Scrollbar(janela_detalhe,orient=tk.VERTICAL,command=tree.yview)
    scrollbar_y.pack(side=tk.RIGHT,fill=tk.Y)

    tree.configure(yscroll=scrollbar_y.set)
    tree.pack(pady=10,expand=True,fill=tk.BOTH)

    for linha in dados[1:]:
        tree.insert('', 'end', values=linha)

    def preencher_campos(event):
        item = tree.focus()
        values = tree.item(item,"values")

        for campo,entry in entries.items():
            entry.delete(0,tk.END)
            entry.insert(0,values[campos.index(campo)])

    tree.bind("<Double-1>",preencher_campos)
    
    def deletar_reserva(tree):
        cpf = entries["CPF"].get()
        data = entries["Data"].get()
        hora_inicio = entries["Hora Inicio"].get()
        hora_fim = entries["Hora Fim"].get()

        if not cpf or not data or not hora_inicio or not hora_fim:
            label_status["text"] = "Por favor, preencha todos os campos"
            return
        
        caminho = "Dados.xlsx"
        panilha = "Reservas"
        try:
            workbook = openpyxl.load_workbook(caminho)
            sheet = workbook[panilha]

        except FileExistsError:
            label_status["text"] = "Arquivo dados não foi encontrado!"
            return

        reservas = sheet.iter_rows(values_only=True)
        next(reservas)

        reserva_encontrada = False
        for i, reservas in enumerate(reservas,start=2):
            if str(reservas[3]) == cpf and str(reservas[0]) == data and str(reservas[1]) == hora_inicio and str(reservas[2]) == hora_fim:
                sheet.delete_rows(i)
                reserva_encontrada = True
                break
        
        if reserva_encontrada:
            try:
                workbook.save(caminho)
                label_status["text"] = f"Reserva excluída com sucesso"
                tree.delete(*tree.get_children())

                dados = sheet.iter_rows(values_only=True)
                next(dados)

                for row in dados:
                    tree.insert("",tk.END,values=row)

                limpar_campos(tree,dados)

            except Exception as e:
                label_status["text"] = f"Erro ao salvar o arquivo: {e}"
        else:
            label_status["text"] = "A reserva informada não foi encontrada"

    def limpar_campos(tree,dados):
        for entry in entries.values():
            entry.delete(0,tk.END)
        
        return dados

    botao_deletar = tk.Button(janela_detalhe,
    font=("Arial",14),text="Deletar Reserva",
    command=lambda:deletar_reserva(tree),width=60)
    botao_deletar.pack(side="left",padx=10,pady=10)

    def exporta_excel(tree):
        caminho_destino = "DadosExportados.xlsx"
        panilha_destino = "Reservas Exportadas"

        try:
            workbook_destino = openpyxl.Workbook()
            sheet_destino = workbook_destino.active
            sheet_destino.title = panilha_destino
            cabecalhos = [tree.heading(column)["text"] for column in tree["columns"]]

            for col, cabecalho in enumerate(cabecalhos,start=1):
                sheet_destino.cell(row=1,column=col,value=cabecalho)

            valores = [tree.item(item)["values"] for item in tree.get_children()]
            
            for row, valor in enumerate(valores,start=2):
                for col, valor in enumerate(valor,start=1):
                    sheet_destino.cell(row=row,column=col,value=valor)

            workbook_destino.save(caminho_destino)

            messagebox.showinfo("Exportação concluida",f"Dados foram exportados para o arquivo {caminho_destino}")

        
        except Exception as e:
            messagebox.showerror("Error",f"Ocorreu um erro ao exportar os dados para excel: {e}")

            label_status["text"] = "Arquivo não encontrado!"
            return
                
    botao_exporta = tk.Button(janela_detalhe,
    font=("Arial",14),text="Exportar",command=lambda:exporta_excel(tree),width=60)
    botao_exporta.pack(side="left",padx=10,pady=10)
    
    janela_detalhe.update_idletasks()

    largura = janela_detalhe.winfo_width()
    altura = janela_detalhe.winfo_height()

    pos_x = (janela_detalhe.winfo_screenwidth() // 2) - (largura // 2)
    pos_y = (janela_detalhe.winfo_screenheight() // 2) - (altura // 2)

    janela_detalhe.geometry(f"{largura}x{altura}+{pos_x}+{pos_y}")    
        
def tela_inicial():
    janela_principal = tk.Tk()
    janela_principal.title("Sistema de Reserva de Quadra")
    janela_principal.configure(bg="#ffffff")

    fonte_titulo = ("Arial",24,"bold")
    fonte_botoes = ("Arial",20)

    tk.Label(janela_principal,
                            text="Sistema de Reservas",
                            font=fonte_titulo,
                            bg="#ffffff").pack(pady=10)

    frame_botoes = tk.Frame(janela_principal,bg="#ffffff")
    frame_botoes.pack(pady=10)

    data_atual = date.today().strftime("%d/%m/%Y")

    botao_fazer_reserva = tk.Button(frame_botoes,
                                text="Fazer Reserva",
                                font=fonte_botoes,
                                command=lambda:fazer_reserva(data_atual),
                                width=30)
    botao_fazer_reserva.pack(pady=10,padx=10)

    botao_ver_reserva = tk.Button(frame_botoes,
                                text="Ver Reservas",
                                font=fonte_botoes,
                                command=ver_reserva,
                                width=30)
    botao_ver_reserva.pack(pady=10)
    
    botao_detalhe_reserva = tk.Button(frame_botoes,
                                text="Datalhe Reservas",
                                font=fonte_botoes,
                                command=detalhe_reserva,
                                width=30)
    botao_detalhe_reserva.pack(pady=10)

    botao_sair = tk.Button(frame_botoes,
                            text="Sair",
                            font=fonte_botoes,
                            command=janela_principal.destroy,
                            width=30)
    botao_sair.pack(pady=10)

    janela_principal.update_idletasks()

    largura = janela_principal.winfo_width()
    altura = janela_principal.winfo_height()

    pos_x = (janela_principal.winfo_screenwidth() // 2) - (largura // 2)
    pos_y = (janela_principal.winfo_screenheight() // 2) - (altura // 2)

    janela_principal.geometry(f"{largura}x{altura}+{pos_x}+{pos_y}")

    janela_principal.mainloop()

tela_inicial()